# Fine-tuning a Model for Summarization Task

In this task, you will load, preprocess, and fine-tune a T5 model on a dataset of news articles for a summarization task. Follow the steps below carefully.

### Model and Dataset Information

For this task, you will be working with the following:

- **Model Checkpoint**: Use the pre-trained model checkpoint `UBC-NLP/AraT5-base` if you face any problem you can use `google-t5/t5-small` but the first one is the correct one for both the model and tokenizer.
- **Dataset**: You will be using the `CUTD/news_articles_df` dataset. Ensure to load and preprocess the dataset correctly for training and evaluation.

**Note:**
- Any additional steps or methods you include that improve or enhance the results will be rewarded with bonus points if they are justified.
- The steps outlined here are suggestions. You are free to implement alternative methods or approaches to achieve the task, as long as you explain the reasoning and the process at the bottom of the notebook.
- You can use either TensorFlow or PyTorch for this task. If you prefer TensorFlow, feel free to use it when working with Hugging Face Transformers.
- The number of data samples you choose to work with is flexible. However, if you select a very low number of samples and the training time is too short, this could affect the evaluation of your work.

## Step 1: Load the Dataset

Load the dataset and split it into training and test sets. Use 20% of the data for testing.

In [1]:
# source https://huggingface.co/datasets/CUTD/news_articles_df
import pandas as pd
df = pd.read_csv("hf://datasets/CUTD/news_articles_df/news_articles_df.csv")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [2]:
from sklearn.model_selection import train_test_split

train_df , test_df = train_test_split(df, test_size=0.2 , random_state=42)

In [3]:
!pip install datasets

In [4]:
from datasets import Dataset

train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)

## Step 2: Load the Pretrained Tokenizer

Initialize a tokenizer from the gevin model checkpoint.

In [5]:
# source : https://huggingface.co/yalsaffar/mt5-small-Arabic-Summarization
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("yalsaffar/mt5-small-Arabic-Summarization")

## Step 3: Preprocess the Dataset

Define a preprocessing function that adds a prefix ("summarize:") to each input if needed and tokenizes the text for the model. The labels will be the tokenized summaries.

In [6]:
#source : https://huggingface.co/docs/transformers/tasks/summarization

prefix = "summarize: "

def preprocess_function(examples):
    inputs = [prefix + doc for doc in examples["text"]]
    model_inputs = tokenizer(inputs, max_length=1024, truncation=True)

    labels = tokenizer(text_target=examples["summarizer"], max_length=128, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

tokenized_train = train_dataset.map(preprocess_function, batched=True)
tokenized_test = test_dataset.map(preprocess_function,batched=True)

Map:   0%|          | 0/6702 [00:00<?, ? examples/s]

Map:   0%|          | 0/1676 [00:00<?, ? examples/s]

## Step 4: Define the Data Collator

Use a data collator designed for sequence-to-sequence models, which dynamically pads inputs and labels.

In [7]:
# source : https://huggingface.co/docs/transformers/main_classes/data_collator#transformers.DataCollatorForSeq2Seq
from transformers import DataCollatorForSeq2Seq
data_collator = DataCollatorForSeq2Seq(tokenizer, model=None,padding=True)

## Step 5: Load the Pretrained Model

Load the model for sequence-to-sequence tasks (summarization).

In [8]:
model = AutoModelForSeq2SeqLM.from_pretrained("yalsaffar/mt5-small-Arabic-Summarization")

## Step 6: Define Training Arguments

Set up the training configuration with parameters like learning rate, batch size, and number of epochs.

## Step 7: Initialize the Trainer

Use the `Seq2SeqTrainer` class to train the model.

In [9]:
#from transformers import TrainingArguments, Trainer

#trainer1 = Trainer(
 #   model=model,
 #   train_dataset=tokenized_train,
 #   eval_dataset=tokenized_test,
 #   tokenizer=tokenizer,
 #   data_collator=data_collator,)
#trainer1.train()

## Step 8: Fine-tune the Model

In [10]:
from transformers import TrainingArguments, Trainer
training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=1,
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    weight_decay=0.01,
    logging_dir="./logs",)

In [11]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_test,
    tokenizer=tokenizer,
    data_collator=data_collator,)
trainer.train()

Step,Training Loss
500,2.038900


Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 128, 'num_beams': 15, 'length_penalty': 0.6, 'no_repeat_ngram_size': 2}
Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 128, 'num_beams': 15, 'length_penalty': 0.6, 'no_repeat_ngram_size': 2}


TrainOutput(global_step=838, training_loss=1.9688222516408114, metrics={'train_runtime': 617.7739, 'train_samples_per_second': 10.849, 'train_steps_per_second': 1.356, 'total_flos': 2918946145075200.0, 'train_loss': 1.9688222516408114, 'epoch': 1.0})

Train the model using the specified arguments and dataset.

## Step 9: Inference

Once the model is trained, perform inference on a sample text to generate a summary. Use the tokenizer to process the text, and then feed it into the model to get the generated summary.

In [20]:
trainer.save_model('./fine-tune')

Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 128, 'num_beams': 15, 'length_penalty': 0.6, 'no_repeat_ngram_size': 2}


In [21]:
tokenizer.save_pretrained('./Tokenize')

('./Tokenize/tokenizer_config.json',
 './Tokenize/special_tokens_map.json',
 './Tokenize/spiece.model',
 './Tokenize/added_tokens.json',
 './Tokenize/tokenizer.json')

In [14]:
train_dataset['text'][296]

'تمكنت الوحدات الامنيه التابعه لمنطقه الامن الوطني بجندوبه بالتعاون فرقه الشرطه العدليه بجندوبه وبعد افاده ايقاف عنصرين يبلغان العمر سنه بتهمه الانتماء جماعه ارهابيه وحسبما مصدر امني لمراسل الجوهره بالجهه المشتبه طالب الثاني جامعه وعاطل العمل مداهمه منزلهما فجر اليوم السبت استشاره النيابه العموميه وتم العثور لديهما كتب الارهاب تم تحويلهما تونس العاصمه لعرضهما انظار القطب القضائي المصدر'

In [22]:
text = "تمكنت الوحدات الامنيه التابعه لمنطقه الامن الوطني بجندوبه بالتعاون فرقه الشرطه العدليه بجندوبه وبعد افاده ايقاف عنصرين يبلغان العمر سنه بتهمه الانتماء جماعه ارهابيه وحسبما مصدر امني لمراسل الجوهره بالجهه المشتبه طالب الثاني جامعه وعاطل العمل مداهمه منزلهما فجر اليوم السبت استشاره النيابه العموميه وتم العثور لديهما كتب الارهاب تم تحويلهما تونس العاصمه لعرضهما انظار القطب القضائي المصدر"

tokenized_text = tokenizer(text, return_tensors="pt").to('cuda')
summary = model.generate(**tokenized_text)

print(tokenizer.decode(summary[0], skip_special_tokens=True))

تمكنت الوحدات الأمنية التابعة للمنطقة الامن الوطني بجندوبة وبعد افاده ايقاف عنصرين يبلغان العمر 16 عاما.
